# Vanilla wikitext generation with pre-trained model

In [29]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en')

In [30]:
DATA_PATH=Path('data/')
PATH = DATA_PATH/'aclImdb'

BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag


We are going to first load a pre-trained language model. Our source LM is the wikitext103 LM created by Stephen Merity @ Salesforce research. [Link to dataset](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/).
The language model for wikitext103 (AWD LSTM) has been pre-trained and the weights can be downloaded here: [Link](http://files.fast.ai/models/wt103/). Our target LM is the style LM. 

In [31]:
# ! wget -nH -r -np -P {PATH} http://files.fast.ai/models/wt103/

The pre-trained LM weights have an embedding size of 400, 1150 hidden units and just 3 layers. We need to match these values  with the target IMDB LM so that the weights can be loaded up.

In [32]:
em_sz,nh,nl = 400,1150,3

In [33]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [34]:
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [35]:
itos2 = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

Let's make a langauge model with no data so we can just play with the generic model.

In [36]:
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [37]:
trn_lm = np.array([[0]])
val_lm = np.array([[0]])

print(val_lm.shape)

vs=len(itos2)

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

(1, 1)


In [38]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [39]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.model.load_state_dict(wgts)

In [40]:
s = "The movie was"
ss = s.lower().split()
si = [stoi2[w] for w in ss]
print(ss)
print(si)

m=learner.model
t=torch.autograd.Variable(torch.LongTensor(np.array([si])))

['the', 'movie', 'was']
[2, 1095, 12]


In [41]:
m[0].bs=1  # Set batch size to 1
m.eval()  # Turn off dropout
m.reset()  # Reset hidden state
res,*_ = m(t)  # Get predictions from model
m[0].bs=bs  # Put the batch size back to what it was

#nexts = torch.topk(res[-1], 10)[1]
#nexts = torch.multinomial(res[-1].exp(), 10)
nexts = res[-1].topk(10)[1]

for i in nexts.data:
    print(itos2[i])

not
the
a
also
used
one
given
considered
an
chosen


In [42]:
m[0].bs=1  # Set batch size to 1
m.eval()  # Turn off dropout
m.reset()  # Reset hidden state
res,*_ = m(t)  # Get predictions from model
m[0].bs=bs  # Put the batch size back to what it was

print(s,"\n")
for i in range(50):
    n = res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(itos2[n], end=' ')
    res,*_ = m(n.unsqueeze(0).unsqueeze(0))
print('...')

The movie was 

not only the first of the two , but also the first to be named after the first person . the second was the " u_n " , which was a reference to the " u_n " of the first world war . the second was the " u_n " ...
